In [240]:
import os
import numpy as np
import pandas as pd

## 실제 호우로 인한 출동 데이터와의 비교
- 2024년 6~7월 간 실제 호우로 인해 발생한 출동데이터 취합
- 학습모델을 통해 예측한 지역별 출동확률과 비교
  - 전체 실제 출동 중 확률이 20% 이상인 곳에 얼마나 많이 걸치나 측정
  - 전체 실제 출동 중에 각 시군구별로 확률이 상위 4개의 지역에 실제 출동이 얼마나 많이 걸치나 측정
  - 전체 실제 출동 중에 각 시군구별로 확률이 상위 4개의 지역에 실제 출동이 얼마나 많이 걸치나 측정

In [241]:
# 2024년 지역별 호우 출동데이터 불러오기
sectors = os.listdir('data/2024_real_rescue')
columns = [
    '기상특보(선택)', '도', '시군', '읍면동', '사망(명)'
]

df_sector_list = []

for sector in sectors:
    path = 'data/2024_real_rescue/' + sector
    df_sector = pd.read_excel(path, sheet_name=None, engine='openpyxl')['Sheet1']
    if '동남구' in sector:
        df_sector['시군'] = df_sector['시군'] + ' ' + '동남구'
    if '서북구' in sector:
        df_sector['시군'] = df_sector['시군'] + ' ' + '서북구'
    df_sector_list.append(df_sector.loc[:, columns])

df_2024_rescue = pd.concat(df_sector_list)
df_2024_rescue.reset_index(drop=True, inplace=True)
df_2024_rescue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기상특보(선택)  819 non-null    object
 1   도         990 non-null    object
 2   시군        990 non-null    object
 3   읍면동       990 non-null    object
 4   사망(명)     309 non-null    object
dtypes: object(5)
memory usage: 38.8+ KB


In [242]:
# 지역별 출동횟수 세기
df_2024_rescue['기상특보(선택)'].fillna('없음', inplace=True)
df_2024_rescue['사망(명)'].fillna(0, inplace=True)
df_2024_rescue['법정동명'] = df_2024_rescue['도'] + ' ' + df_2024_rescue['시군'] + ' ' + df_2024_rescue['읍면동']
# df_2024_rescue['법정동명'] = ''
# df_2024_rescue['법정동코드'] = 0
# df_2024_rescue.rename(columns={
#     '기상특보(선택)': 'WARNING', 
# })
df_2024_rescue = df_2024_rescue.groupby('법정동명').count().reset_index()
df_2024_rescue = df_2024_rescue.loc[:, ['법정동명', '기상특보(선택)']]
df_2024_rescue.rename(columns={'기상특보(선택)': '출동횟수'}, inplace=True)
df_2024_rescue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동명    192 non-null    object
 1   출동횟수    192 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ KB


In [244]:
# 법정동코드 데이터 불러오기 후 읍면동 단위로 정제
law_code = pd.read_table("data/법정동코드 전체자료.txt", sep='\t', encoding='cp949')
law_code = law_code[law_code['법정동코드'].apply(lambda x: str(x).startswith('44'))]
law_code = law_code[law_code['폐지여부'] == '존재']
law_code = law_code[law_code['법정동명'].apply(lambda x: x[-1] in ('읍', '면', '동'))]
law_code.loc[:, '법정동코드'] = law_code.loc[:, '법정동코드'].apply(lambda x: str(x)[:-2])
law_code.reset_index(drop=True, inplace=True)
law_code.drop(columns='폐지여부', inplace=True)
print(len(law_code))
law_code.head(3)

285


,법정동코드,법정동명
0,44131101,충청남도 천안시 동남구 대흥동
1,44131102,충청남도 천안시 동남구 성황동
2,44131103,충청남도 천안시 동남구 문화동


In [245]:
# 출동횟수 데이터랑 법정동코드 데이터랑 merge
df_rescue = pd.merge(law_code, df_2024_rescue, how='outer', on='법정동명')
df_rescue['출동횟수'].fillna(0, inplace=True)
df_rescue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294 entries, 0 to 293
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   법정동코드   285 non-null    object 
 1   법정동명    294 non-null    object 
 2   출동횟수    294 non-null    float64
dtypes: float64(1), object(2)
memory usage: 9.2+ KB


In [246]:
# 법정동코드가 결측치인 행 찾기
df_rescue[df_rescue['법정동코드'].isna()]

,법정동코드,법정동명,출동횟수
285,NaN,충청남도 금산군 금산읍,1.0
286,NaN,충청남도 논산시 양촌면,9.0
287,NaN,충청남도 논산시 연무읍,1.0
288,NaN,충청남도 논산시 엽무읍,1.0
289,NaN,충청남도 부여군 귱암면,1.0
290,NaN,충청남도 서천군 한산읍,1.0
291,NaN,충청남도 천안시 동남구 목천읍,1.0
292,NaN,충청남도 천안시 동남구 신부동,4.0
293,NaN,충청남도 천안시 서북구 다가동,1.0


In [247]:
# 법정동명에 공백이 포함되거나 오타가 있는 경우 법정동코드에 결측치가 생김
# 개수가 적으므로 하나씩 직접 수정
df_rescue.loc[285, '법정동명'] = df_rescue.loc[285, '법정동명'].rstrip()
df_rescue.loc[286, '법정동명'] = df_rescue.loc[286, '법정동명'].rstrip()
df_rescue.loc[287, '법정동명'] = df_rescue.loc[287, '법정동명'].rstrip()
df_rescue.loc[288, '법정동명'] = '충청남도 논산시 연무읍'
df_rescue.loc[289, '법정동명'] = '충청남도 부여군 규암면'
df_rescue.loc[290, '법정동명'] = '충청남도 서천군 한산면'
df_rescue.loc[291, '법정동명'] = df_rescue.loc[291, '법정동명'].rstrip()
df_rescue.loc[292, '법정동명'] = df_rescue.loc[292, '법정동명'].rstrip()
df_rescue.loc[293, '법정동명'] = '충청남도 천안시 동남구 다가동'

In [248]:
# 수정 이후 법정동코드별로 출동횟수 합하기
df_rescue = df_rescue.groupby(['법정동코드', '법정동명']).sum().reset_index()
df_rescue.rename(columns={'법정동코드': 'EMD_CD', '법정동명': 'EMD_KOR_NM', '출동횟수': 'RES'}, inplace=True)
df_rescue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   EMD_CD      285 non-null    object 
 1   EMD_KOR_NM  285 non-null    object 
 2   RES         285 non-null    float64
dtypes: float64(1), object(2)
memory usage: 6.8+ KB


In [249]:
# 지역별 출동예측확률 불러오기
import geopandas as gpd

columns = ['EMD_CD', 'EMD_KOR_NM', 'RESCUE_PROB', 'DEATH', 'DAM']

df_predict = gpd.read_file('./web_data/emd_pp.json', encoding='cp949').loc[:, columns]
df_predict.head()

,EMD_CD,EMD_KOR_NM,RESCUE_PROB,DEATH,DAM
0,44230400,충청남도 논산시 가야곡면,0.334279,0,0
1,44210103,충청남도 서산시 갈산동,0.286383,0,0
2,44800380,충청남도 홍성군 갈산면,0.057232,0,0
3,44230250,충청남도 논산시 강경읍,0.268393,0,1
4,44230110,충청남도 논산시 강산동,0.096039,0,0


In [256]:
# 출동예측확률과 실제 출동발생횟수 merge
df_final = pd.merge(df_rescue, df_predict, how='inner', on=['EMD_CD', 'EMD_KOR_NM'])
df_final.head()

,EMD_CD,EMD_KOR_NM,RES,RESCUE_PROB,DEATH,DAM
0,44131101,충청남도 천안시 동남구 대흥동,1.0,0.176888,0,0
1,44131102,충청남도 천안시 동남구 성황동,0.0,0.118649,0,0
2,44131103,충청남도 천안시 동남구 문화동,0.0,0.510843,0,0
3,44131104,충청남도 천안시 동남구 사직동,0.0,0.403644,0,0
4,44131105,충청남도 천안시 동남구 영성동,1.0,0.179828,0,0


In [297]:
# 전체 실제 출동 중 확률이 20% 이상인 곳에 얼마나 많이 걸치나 측정
df_final_20 = df_final[df_final['RESCUE_PROB'] >= 0.20]
print('전체 출동횟수:', int(df_final['RES'].sum()))
print('초록부분에 걸치는 출동횟수:', int(df_final_20['RES'].sum()))
print('비율:', df_final_20['RES'].sum() / df_final['RES'].sum())

전체 출동횟수: 970
초록부분에 걸치는 출동횟수: 337
비율: 0.3474226804123711


In [273]:
# 시군구별 구분을 위해 시군구 컬럼 추가
df_final['시군구'] = df_final['EMD_KOR_NM'].apply(
    lambda x: x.split()[1] + " " + x.split()[2] if len(x.split()) == 4 else x.split()[1]
)

df_final.head()

,EMD_CD,EMD_KOR_NM,RES,RESCUE_PROB,DEATH,DAM,시군구
0,44131101,충청남도 천안시 동남구 대흥동,1.0,0.176888,0,0,천안시 동남구
1,44131102,충청남도 천안시 동남구 성황동,0.0,0.118649,0,0,천안시 동남구
2,44131103,충청남도 천안시 동남구 문화동,0.0,0.510843,0,0,천안시 동남구
3,44131104,충청남도 천안시 동남구 사직동,0.0,0.403644,0,0,천안시 동남구
4,44131105,충청남도 천안시 동남구 영성동,1.0,0.179828,0,0,천안시 동남구


In [298]:
# 전체 실제 출동 중에 각 시군구별로 확률이 상위 4개의 지역에 실제 출동이 얼마나 많이 걸치나 측정
total_rescue = 0

for s in set(df_final['시군구'].values.tolist()):
    df_s = df_final[df_final['시군구'] == s].sort_values('RESCUE_PROB')[::-1][:3]
    total_rescue += df_s['RES'].sum()


print('전체 출동횟수:', int(df_final['RES'].sum()))
print('시군구별 상위 3지역에 걸치는 출동횟수:', int(total_rescue))
print('비율:', total_rescue / df_final['RES'].sum())

전체 출동횟수: 970
시군구별 상위 3지역에 걸치는 출동횟수: 184
비율: 0.18969072164948453


In [299]:
# 전체 실제 출동 중에 각 시군구별로 확률이 상위 4개의 지역에 실제 출동이 얼마나 많이 걸치나 측정
total_rescue = 0

for s in set(df_final['시군구'].values.tolist()):
    df_s = df_final[df_final['시군구'] == s].sort_values('RESCUE_PROB')[::-1][:4]
    total_rescue += df_s['RES'].sum()

print('전체 출동횟수:', int(df_final['RES'].sum()))
print('시군구별 상위 4지역에 걸치는 출동횟수:', int(total_rescue))
print('비율:', total_rescue / df_final['RES'].sum())

전체 출동횟수: 970
시군구별 상위 4지역에 걸치는 출동횟수: 251
비율: 0.25876288659793817
